In [ ]:
import sys
import numpy as np
import keras
import random


path = keras.utils.get_file(
    'romeo-and-juliet_TXT_FolgerShakespeare.txt',
    origin='https://folger-main-site-assets.s3.amazonaws.com/uploads/2022/11/romeo-and-juliet_TXT_FolgerShakespeare.txt' 
    
)
text = open(path, encoding='utf-8').read().lower()


chars = sorted(list(set(text)))
print('Numero di caratteri unici:', len(chars))

char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}


maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

print('Numero di sequenze:', len(sentences))


def batch_generator(sentences, next_chars, char_indices, maxlen, batch_size=128):
    while True:
        for i in range(0, len(sentences), batch_size):
            current_batch_size = min(batch_size, len(sentences) - i)
            x = np.zeros((current_batch_size, maxlen, len(char_indices)), dtype=np.float32)
            y = np.zeros((current_batch_size, len(char_indices)), dtype=np.float32)
            for j, sentence in enumerate(sentences[i:i+batch_size]):
                for t, char in enumerate(sentence):
                    x[j, t, char_indices[char]] = 1.0
                y[j, char_indices[next_chars[i+j]]] = 1.0
            yield x, y


model = keras.Sequential([
    keras.layers.Input(shape=(maxlen, len(chars))),
    keras.layers.LSTM(128),
    keras.layers.Dense(len(chars), activation='softmax')
])
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(np.clip(preds, 1e-8, 1)) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


batch_size = 128
steps_per_epoch = len(sentences) // batch_size

for epoch in range(1, 61):  
    print(f"\nEpoch {epoch}")
    model.fit(
        batch_generator(sentences, next_chars, char_indices, maxlen, batch_size),
        steps_per_epoch=steps_per_epoch,
        epochs=1
    )

  
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('\n-- Diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                if char in char_indices:
                    x_pred[0, t, char_indices[char]] = 1
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()


Numero di caratteri unici: 52
Numero di sequenze: 47212

Epoch 1
368/368 ━━━━━━━━━━━━━━━━━━━━ 13s 32ms/step - accuracy: 0.3004 - loss: 2.4576

----- Diversity: 0.2
----- Generating with seed: "that living mortals, hearing them, run mad--
o, if i wake, s"
that living mortals, hearing them, run mad--
o, if i wake, she pring of the pris the pringed and her dith me the pring and my live
the pring this love the parth me pring stour whis lath my live mure the souns.

princh
and she print whis the pring the pring the sill the pring the priar the sound she pring the sill whe pring the pring the prear ho the lath the the pring the sount
the pring the sound the pring the sare the the pring the pring i whin my lath t
----- Diversity: 0.5
----- Generating with seed: "that living mortals, hearing them, run mad--
o, if i wake, s"
that living mortals, hearing them, run mad--
o, if i wake, share this frin that hour on hous me this and bar julled she love my and i and what for sher and hit with and thi